## Improving on Script
* Feedback: https://github.com/cal-itp/data-analyses/pull/961

In [7]:
import datetime

import dask
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.geography_utils import WGS84
from loguru import logger
from scripts import vp_spatial_accuracy
from segment_speed_utils import helpers, wrangle_shapes
from segment_speed_utils.project_vars import (
    GCS_FILE_PATH,
    PROJECT_CRS,
    SEGMENT_GCS,
    analysis_date,
)

In [8]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [9]:
from scripts import update_vars2

In [10]:
update_vars2.analysis_date_list

['2023-11-15',
 '2023-10-11',
 '2023-09-13',
 '2023-08-15',
 '2023-07-12',
 '2023-06-14',
 '2023-05-17',
 '2023-04-12',
 '2023-03-15']

### Read in files

In [11]:
may_df = pd.read_parquet('gs://calitp-analytics-data/data-analyses/rt_vs_schedule/trip_level_metrics/2023-05-17_metrics.parquet')

In [12]:
may_df.shape

(83606, 15)

In [14]:
may_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83606 entries, 0 to 83605
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   trip_instance_key             83606 non-null  object 
 1   rt_service_min                83606 non-null  float64
 2   min_w_atleast2_trip_updates   83606 non-null  int64  
 3   total_pings_for_trip          83606 non-null  int64  
 4   total_min_w_gtfs              83606 non-null  int64  
 5   total_vp                      65385 non-null  float64
 6   vp_in_shape                   65385 non-null  float64
 7   speed_mph                     67864 non-null  float64
 8   route_id                      66947 non-null  object 
 9   time_of_day                   67864 non-null  object 
 10  service_minutes               67864 non-null  float64
 11  pings_per_min                 83606 non-null  float64
 12  spatial_accuracy_pct          65385 non-null  float64
 13  r

In [15]:
may_df.sample(10)

,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings_for_trip,total_min_w_gtfs,total_vp,vp_in_shape,speed_mph,route_id,time_of_day,service_minutes,pings_per_min,spatial_accuracy_pct,rt_w_gtfs_pct,rt_v_scheduled_trip_time_pct
67705,5935683f1de5971e798ff46df16aa992,51.28,51,152,51,152.00,152.00,2.50,30,Midday,36.00,2.96,100.00,99.45,42.45
46626,3aa68fa530c1ea648e8b71f5b101fbd3,34.50,27,57,30,57.00,57.00,9.43,4,PM Peak,30.00,1.65,100.00,86.96,15.00
61276,b63485dcea31cef202434b49279b9959,136.35,134,380,136,380.00,317.00,7.94,301,AM Peak,110.00,2.79,83.42,99.74,23.95
65371,185682061431ba6a11af70b359c3ebb2,46.20,46,136,46,136.00,136.00,5.53,14R,Evening,51.00,2.94,100.00,99.57,-9.41
13656,f33622685e5eda4c247561ac98800bdc,67.92,67,202,68,202.00,202.00,15.49,None,Midday,60.00,2.97,100.00,100.12,13.19
83045,f876d064d2030c6c63f111a295b7db6e,33.25,33,98,33,98.00,98.00,13.37,233,PM Peak,24.00,2.95,100.00,99.25,38.54
59788,6ada7d1db0c8eaf701657992b0a6f9fd,57.52,57,171,57,171.00,160.00,14.47,b6bc2958-333f-4d1e-8579-b75b2698b6b4,Midday,35.00,2.97,93.57,99.10,64.33
76087,03bfccb0ce4af0600c28e67160cd402f,74.48,74,216,75,NaN,NaN,4.34,20,Midday,62.00,2.90,NaN,100.69,20.13
24358,59a4494ae6b581ff202eda7ca1399fb3,72.93,73,214,73,214.00,177.00,10.79,240-13167,Early AM,57.00,2.93,82.71,100.09,27.95
34274,da177e6b35ccf7c82c4933dd3248a465,25.97,25,76,26,NaN,NaN,NaN,None,None,NaN,2.93,NaN,100.13,NaN


### Filter columns 
* ['trip_instance_key', 'location_timestamp_local', 'x','y','vp_idx']

In [ ]:
operator = "Bay Area 511 Muni VehiclePositions"
gtfs_key = "7cc0cb1871dfd558f11a2885c145d144"

In [ ]:
def load_vp_usable(analysis_date):

    # Delete schedule_gtfs_dataset_key later
    df = dd.read_parquet(
        f"{SEGMENT_GCS}vp_usable_{analysis_date}",
        columns=[
            "schedule_gtfs_dataset_key",
            "trip_instance_key",
            "location_timestamp_local",
            "x",
            "y",
            "vp_idx",
        ],
    )

    # Create a copy of location timestamp for the total_trip_time function
    # to avoid type setting
    df["max_time"] = df.location_timestamp_local
    return df

In [ ]:
vp_usable = load_vp_usable(analysis_date)
# Filter for now
vp_usable2 = vp_usable.loc[vp_usable.schedule_gtfs_dataset_key == gtfs_key].reset_index(
    drop=True
)

In [ ]:
# vp_usable2 = vp_usable2.compute()

In [ ]:
type(vp_usable2)

### Total Trip Time
* Addresses "<i>in this function, min_time, max_time are created on the grouped df (vp_usable grouped by trip and binned minute)...I think to be safer, it should be created on vp_usable grouped by trip.</i>"
* The copy setting is now turned on?? How to get it to go away?

In [ ]:
def total_trip_time(vp_usable_df: pd.DataFrame):
    """
    For each trip: find the total service minutes
    recorded in real time data so we can compare it with
    scheduled service minutes.
    """
    subset = ["location_timestamp_local", "trip_instance_key", "max_time"]
    vp_usable_df = vp_usable_df[subset]

    # Need an extra copy of the column to find the max

    # Find the max and the min time based on location timestamp
    df = (
        vp_usable_df.groupby(["trip_instance_key"])
        .agg({"location_timestamp_local": "min", "max_time": "max"})
        .reset_index()
        .rename(columns={"location_timestamp_local": "min_time"})
    )

    # Find total rt service mins and add an extra minute
    df["rt_service_min"] = (df.max_time - df.min_time) / pd.Timedelta(minutes=1) + 1

    # Return only one row per trip with the total trip time
    df = df.drop(columns=["max_time", "min_time"])

    return df

#### Change in script: remove map partitions

In [ ]:
start = datetime.datetime.now()
print(start)
test = total_trip_time(vp_usable)

end = datetime.datetime.now()
logger.info(f"execution time: {end-start}")

In [ ]:
# total_trip_time_df = total_trip_time_df.compute()

In [ ]:
test.head()

### Update Completeness
#### Break apart?

In [ ]:
def trips_by_one_min(vp_usable_df: pd.DataFrame):
    """
    For each trip: count how many rows are associated with each minute
    then tag whether or not a minute has 2+ pings. 
    """
    subset = ["location_timestamp_local", "trip_instance_key", "vp_idx"]
    vp_usable_df = vp_usable_df[subset]

    # Find number of pings each minute
    df = (
        vp_usable_df.groupby(
            [
                "trip_instance_key",
                pd.Grouper(key="location_timestamp_local", freq="1Min"),
            ]
        )
        .vp_idx.count()
        .reset_index()
        .rename(columns={"vp_idx": "number_of_pings_per_minute"})
    )

    # Determine which rows have 2+ pings per minute
    df = df.assign(
        min_w_atleast2_trip_updates=df.apply(
            lambda x: 1 if x.number_of_pings_per_minute >= 2 else 0, axis=1
        )
    )
    
    df = df.drop(columns = ['location_timestamp_local'])
    return df

In [ ]:
# map partitions here 
# test = trips_by_min(vp_usable2)

In [ ]:
start = datetime.datetime.now()
print(start)
one_min_ping_df = vp_usable.map_partitions(
    trips_by_one_min,
    meta={
        "trip_instance_key": "object",
        "number_of_pings_per_minute": "int64",
        "min_w_atleast2_trip_updates":"int64"
    },
    align_dataframes=False,
).persist()

end = datetime.datetime.now()
logger.info(f"execution time: {end-start}")

In [ ]:
len(one_min_ping_df)

In [ ]:
type(one_min_ping_df)

In [ ]:
def update_completeness(df: pd.DataFrame):
    """
    For each trip: find the median GTFS pings per minute,
    the total minutes with at least 1 GTFS ping per minute,
    and total minutes with at least 2 GTFS pings per minute.
    """
    # Need a copy of numer of pings per minute to count for total minutes w gtfs
    df["total_min_w_gtfs"] = df.number_of_pings_per_minute
    
    # Find the total min with at least 2 pings per min
    df = (
        df.groupby(["trip_instance_key"])
        .agg(
            {
                "min_w_atleast2_trip_updates": "sum",
                "number_of_pings_per_minute": "sum",
                "total_min_w_gtfs": "count",
            }
        )
        .reset_index()
        .rename(
            columns={
                "number_of_pings_per_minute": "total_pings_for_trip",
            }
        )
    )

    return df

In [ ]:
start = datetime.datetime.now()
print(start)
update_df = update_completeness(one_min_ping_df)
end = datetime.datetime.now()
logger.info(f"execution time: {end-start}")

In [ ]:
update_df

#### Testing new way to find pings per min

In [ ]:
m1 = dd.merge(update_df, test, on = ['trip_instance_key'], how = 'inner')

In [ ]:
m1 = m1.compute()

In [ ]:
m1['ping_per_min'] = (m1.total_pings_for_trip / m1.rt_service_min)

In [ ]:
m1.columns

In [ ]:
full_df = pd.read_parquet("./scripts/rt_v_schedule_trip_metrics.parquet")

In [ ]:
subset = ['trip_instance_key','ping_per_min', 'rt_service_min']

In [ ]:
pings_test = pd.merge(m1[subset], full_df[['trip_instance_key', 'avg_pings_per_min', 'rt_service_min']], on = 'trip_instance_key', how = "inner")

In [ ]:
pings_test['difference'] = pings_test.ping_per_min - pings_test.avg_pings_per_min

In [ ]:
pings_test['difference'].describe()

In [ ]:
pings_test.loc[pings_test.difference < 0].sort_values(by = ['difference'], ascending = True).head(30)

In [ ]:
m1.loc[m1.trip_instance_key == "923ff90bea616b5ac4ec909d79503424"]

In [ ]:
one_trip = one_min_ping_df.loc[one_min_ping_df.trip_instance_key == "923ff90bea616b5ac4ec909d79503424"].compute()

In [ ]:
one_trip_og = vp_usable.loc[vp_usable.trip_instance_key == "923ff90bea616b5ac4ec909d79503424"].compute()

In [ ]:
# one_trip_og

In [ ]:
 # one_trip

In [ ]:
one_trip.number_of_pings_per_minute.mean()

In [ ]:
one_trip.number_of_pings_per_minute.sum()/198.6

### Spatial Accuracy
* Addresses "<i>in next draft, work on grouping functions that belong together, such as this one. total_counts and total_counts_by_trip sound basically equivalent, and they are nearly doing the same thing, except total_counts actually creates 2 columns. work on logically grouping or absorbing functions or rewriting functions so the same function can now be used twice.
Adapt this function to be used twice
Compare it to this to find where they have stuff in common and which part should be removed from the generic function</i>"

In [ ]:
def grab_shape_keys_in_vp(vp_usable: dd.DataFrame, analysis_date: str) -> pd.DataFrame:
    """
    Subset raw vp and find unique trip_instance_keys.
    Create crosswalk to link trip_instance_key to shape_array_key.
    """
    vp_usable = (
        vp_usable[["trip_instance_key"]].drop_duplicates().reset_index(drop=True)
    )

    trips_with_shape = helpers.import_scheduled_trips(
        analysis_date,
        columns=["trip_instance_key", "shape_array_key"],
        get_pandas=True,
    )

    # Only one row per trip/shape
    # trip_instance_key and shape_array_key are the only 2 cols left
    m1 = dd.merge(vp_usable, trips_with_shape, on="trip_instance_key", how="inner")

    return m1

### Why is September not working


In [ ]:
date2= '2023-09-13'

In [ ]:
vp_usable = load_vp_usable(date2)
# Filter for now
vp_usable2 = vp_usable.loc[vp_usable.schedule_gtfs_dataset_key == gtfs_key].reset_index(
    drop=True
)

In [ ]:
vp_usable2 = vp_usable2.compute()

In [ ]:
vp_usable2.schedule_gtfs_dataset_key.unique()

In [ ]:
test_keys = vp_usable2[['trip_instance_key']].drop_duplicates()

In [ ]:
test_keys.shape

In [ ]:
test_keys.head()

In [ ]:
 trips_with_shape = helpers.import_scheduled_trips(
        date2,
        columns=["trip_instance_key", "shape_array_key"],
        get_pandas=True,
    )

In [ ]:
m1 = pd.merge(test_keys, trips_with_shape, on="trip_instance_key", how="inner")

In [ ]:
m1.shape

In [ ]:
shapes_df = grab_shape_keys_in_vp(vp_usable2, date2)

In [ ]:
type(shapes_df)

In [ ]:
shapes_df.shape

In [ ]:
subset = shapes_df.shape_array_key.unique().tolist()

In [ ]:
subset

In [ ]:
def buffer_shapes(
    trips_with_shape: pd.DataFrame,
    analysis_date: str,
    buffer_meters: int = 35,
):
    """
    Filter scheduled shapes down to the shapes that appear in vp.
    Buffer these.

    Attach the shape geometry for a subset of shapes or trips.
    """
    subset = trips_with_shape.shape_array_key.unique().compute().tolist()

    shapes = helpers.import_scheduled_shapes(
        analysis_date,
        columns=["shape_array_key", "geometry"],
        filters=[[("shape_array_key", "in", subset)]],
        crs=PROJECT_CRS,
        get_pandas=False,
    ).pipe(helpers.remove_shapes_outside_ca)

    # to_crs takes awhile, so do a filtering on only shapes we need
    shapes = shapes.assign(geometry=shapes.geometry.buffer(buffer_meters))

    trips_with_shape_geom = dd.merge(
        shapes, trips_with_shape, on="shape_array_key", how="inner"
    )

    trips_with_shape_geom = trips_with_shape_geom.compute()
    return trips_with_shape_geom

In [ ]:
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS

In [ ]:
COMPILED_CACHED_VIEWS

In [ ]:
FILE = f"{COMPILED_CACHED_VIEWS}routelines_{date2}.parquet"

In [ ]:
FILE

In [ ]:
# gs://calitp-analytics-data/data-analyses/rt_delay/compiled_cached_views/routelines_2023-09-13.parquet

In [ ]:
shapes = helpers.import_scheduled_shapes(
        date2,
        columns=["shape_array_key", "geometry"],
        crs=PROJECT_CRS,
        get_pandas=False,
    ).pipe(helpers.remove_shapes_outside_ca)

In [ ]:
import dask_geopandas as dg

In [ ]:
test = dg.read_parquet(FILE)

In [ ]:
test2 = gpd.read_parquet(FILE)

In [ ]:
test2.columns

In [ ]:
buffer_df = buffer_shapes(shapes_df, date2, 35)

In [ ]:
def vp_in_shape(
    vp_usable: dd.DataFrame, trips_with_buffered_shape: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:

    keep = ["trip_instance_key", "x", "y", "location_timestamp_local"]
    vp_usable = vp_usable[keep]

    vp_gdf = wrangle_shapes.vp_as_gdf(vp_usable)

    gdf = pd.merge(
        vp_gdf, trips_with_buffered_shape, on="trip_instance_key", how="inner"
    )

    gdf = gdf.assign(is_within=gdf.geometry_x.within(gdf.geometry_y))
    gdf = gdf[["trip_instance_key", "location_timestamp_local", "is_within"]]

    return gdf

In [ ]:
start = datetime.datetime.now()
print(start)
spatial_accuracy_df1 = vp_usable.map_partitions(
    vp_in_shape,
    buffer_df,
    meta={
        "trip_instance_key": "object",
        "location_timestamp_local": "datetime64[ns]",
        "is_within": "bool",
    },
    align_dataframes=False,
).persist()

end = datetime.datetime.now()
logger.info(f"execution time: {end-start}")

#### Adapt this to be used in multiple places

In [ ]:
def total_vp_counts_by_trip(vp: gpd.GeoDataFrame, new_col_title: str) -> pd.DataFrame:
    """
    Get a count of vp for each trip, whether or not those fall
    within buffered shape or not
    """
    count_vp = (
        vp.groupby("trip_instance_key", observed=True, group_keys=False)
        .agg({"location_timestamp_local": "count"})
        .reset_index()
        .rename(columns={"location_timestamp_local": new_col_title})
    )

    return count_vp

In [ ]:
def total_counts(result: dd.DataFrame):

    # Find the total number of vps for each route
    total_vp_df = total_vp_counts_by_trip(result, "total_vp")

    # Find the total number of vps that actually fall within the  route shape
    subset = ["trip_instance_key", "location_timestamp_local"]
    result2 = result.loc[result.is_within == True].reset_index(drop=True)[subset]

    vps_in_shape = total_vp_counts_by_trip(result2, "vp_in_shape")

    # Count total vps for the trip
    count_df = pd.merge(total_vp_df, vps_in_shape, on="trip_instance_key", how="left")

    count_df = count_df.assign(
        vp_in_shape=count_df.vp_in_shape.fillna(0).astype("int32"),
        total_vp=count_df.total_vp.fillna(0).astype("int32"),
    )

    return count_df

In [ ]:
start = datetime.datetime.now()
print(start)
spatial_accuracy_df2 = spatial_accuracy_df1.map_partitions(
    total_counts,
    meta={"trip_instance_key": "object", "total_vp": "int32", "vp_in_shape": "int32"},
    align_dataframes=False,
).persist()

end = datetime.datetime.now()
logger.info(f"execution time: {end-start}")

In [ ]:
spatial_accuracy_df2 = spatial_accuracy_df2.compute()

In [ ]:
spatial_accuracy_df2.head()

In [ ]:
type(update)

test_m = (total_trip_time_df.merge(update, on = "trip_instance_key", how = "outer")
         .merge(spatial_accuracy_df2, on ="trip_instance_key", how = "outer")) 

### Read back in the file 12/11

In [3]:
full_df = pd.read_parquet("./scripts/rt_v_schedule_trip_metrics.parquet")

In [4]:
full_df.sample()

,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,avg_pings_per_min,total_min_w_gtfs,total_vp,vp_in_shape,speed_mph,route_id,time_of_day,service_minutes
85164,bb0d2159703990dfd08a1d357ae90ff8,461.97,93,2.13,136,290.00,289.00,24.42,801,PM Peak,115.00


In [5]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86832 entries, 0 to 86831
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   trip_instance_key            86832 non-null  object 
 1   rt_service_min               86832 non-null  float64
 2   min_w_atleast2_trip_updates  86832 non-null  int64  
 3   avg_pings_per_min            86832 non-null  float64
 4   total_min_w_gtfs             86832 non-null  int64  
 5   total_vp                     74891 non-null  float64
 6   vp_in_shape                  74891 non-null  float64
 7   speed_mph                    77194 non-null  float64
 8   route_id                     75619 non-null  object 
 9   time_of_day                  77194 non-null  object 
 10  service_minutes              77194 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 7.9+ MB


In [6]:
full_df.trip_instance_key.nunique(), len(full_df)

(86832, 86832)